# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# calculates the totals
school_totals = len(school_data['school_name'].unique())
student_totals = len(student_data['student_name'])
budget_totals = school_data['budget'].sum()
math_totals = student_data['math_score'].mean()
reading_totals = student_data['reading_score'].mean()

# uses bins to passes/fails, counts them and calculates percentages
bins = [0, 69, 100]
group_names = ['fail', 'pass']
reading_rate = student_data.loc[:, ['student_name', 'reading_score']]
math_rate = student_data.loc[:, ['student_name', 'math_score']]
reading_rate["reading_score"] = pd.cut(reading_rate["reading_score"], bins, labels=group_names)
math_rate["math_score"] = pd.cut(math_rate["math_score"], bins, labels=group_names)
reading_rate = reading_rate.groupby('reading_score')
math_rate = math_rate.groupby('math_score')
reading_rate = reading_rate.count()
math_rate = math_rate.count()
reading_pass = (reading_rate.iat[1, 0]/reading_rate['student_name'].sum())*100
math_pass = (math_rate.iat[1, 0]/math_rate['student_name'].sum())*100
math_reading_pass = (math_pass+reading_pass)/2

# create a dataframe to display the output
distSum = {'Total Schools': [school_totals],
          'Total Students': [student_totals],
          'Total Budget': [budget_totals],
          'Average Math Score': [math_totals],
          'Average Reading Score': [reading_totals],
          'Passing Math': [math_pass],
          'Passing Reading': [reading_pass],
          'Overall Passing Rate': [math_reading_pass]}
distSum = pd.DataFrame(distSum)

# format columns
distSum['Total Budget'] = distSum['Total Budget'].map("${:.2f}".format)
distSum['Average Math Score'] = distSum['Average Math Score'].map("{:.2f}".format)
distSum['Average Reading Score'] = distSum['Average Reading Score'].map("{:.2f}".format)
distSum['Passing Math'] = distSum['Passing Math'].map("{:.2f}%".format)
distSum['Passing Reading'] = distSum['Passing Reading'].map("{:.2f}%".format)
distSum['Overall Passing Rate'] = distSum['Overall Passing Rate'].map("{:.2f}%".format)

distSum

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
0,15,39170,$24649428.00,78.99,81.88,74.98%,85.81%,80.39%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:
# creates a dataframe grouped by school and type using .mean() which gives average scores and the budget and total students
top_schools = school_data_complete.loc[:, ['school_name', 'type', 'size', 'budget', 'math_score', 'reading_score']]
top_schools = top_schools.rename(columns={'school_name': 'School Name', 'type': 'School Type'})
top_school_group = top_schools.groupby(['School Name', 'School Type'])
top_school_group = top_school_group.mean()
# adds a column for the Per Student Budget
top_school_group['Per Student Budget'] = top_school_group['budget']/top_school_group['size']
# uses bins to passes/fails for math and reading
bins = [0, 69, 100]
group_names = ['fail', 'pass']
reading_rate = school_data_complete.loc[:, ['school_name', 'student_name', 'reading_score']]
math_rate = school_data_complete.loc[:, ['school_name', 'student_name', 'math_score']]
reading_rate["reading_score"] = pd.cut(reading_rate["reading_score"], bins, labels=group_names)
math_rate["math_score"] = pd.cut(math_rate["math_score"], bins, labels=group_names)
reading_rate = reading_rate.groupby(['school_name','reading_score'])
math_rate = math_rate.groupby(['school_name','math_score'])
reading_rate = reading_rate.count()
math_rate = math_rate.count()
math_pass = []
math_fail = []
read_pass = []
read_fail = []
i = 1

# loops through math and reading pass/fail bins for each school 
while i < 30:
    math_pass.append(math_rate.iat[i, 0])
    math_fail.append(math_rate.iat[i-1, 0])
    read_pass.append(reading_rate.iat[i, 0])
    read_fail.append(reading_rate.iat[i-1, 0])
    i = i + 2
    
# calculates math pass% for each school and adds it as a column in our dataframe
top_school_group['% Passing Math'] = math_pass
top_school_group['math_fail'] = math_fail
top_school_group['% Passing Math'] = (top_school_group['% Passing Math']/(top_school_group['% Passing Math']+top_school_group['math_fail']))*100
del top_school_group['math_fail']

# calculates reading pass% for each school and adds it as a column in our dataframe
top_school_group['% Passing Reading'] = read_pass
top_school_group['read_fail'] = read_fail
top_school_group['% Passing Reading'] = (top_school_group['% Passing Reading']/(top_school_group['% Passing Reading']+top_school_group['read_fail']))*100
del top_school_group['read_fail']

# calculates overall pass% for each school and adds it as a column in our dataframe
top_school_group['% Overall Passing Rate'] = (top_school_group['% Passing Math']+top_school_group['% Passing Reading'])/2

# renames, sorts, and formats our columns
top_school_group = top_school_group.rename(columns={'size': 'Total Students',
                                        'budget': 'Total School Budget',
                                        'math_score': 'Average Math Score',
                                        'reading_score': 'Average Reading Score'})
top_school_group = top_school_group.loc[:,['Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score',
                            'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']] 
top_school_group = top_school_group.sort_values(by=['% Overall Passing Rate'], ascending=False)
bottom_school_group = top_school_group.sort_values(by=['% Overall Passing Rate'], ascending=True)

# creates unformated dataframe with indexes reset for later use
unformatted_school_group = top_school_group.reset_index()
top_school_group['Total Students'] = top_school_group['Total Students'].map("{:.0f}".format)
top_school_group['Total School Budget'] = top_school_group['Total School Budget'].map("${:.2f}".format)
top_school_group['Average Math Score'] = top_school_group['Average Math Score'].map("{:.2f}".format)
top_school_group['Average Reading Score'] = top_school_group['Average Reading Score'].map("{:.2f}".format)    
top_school_group['% Passing Math'] = top_school_group['% Passing Math'].map("{:.2f}%".format)
top_school_group['% Passing Reading'] = top_school_group['% Passing Reading'].map("{:.2f}%".format)
top_school_group['% Overall Passing Rate'] = top_school_group['% Overall Passing Rate'].map("{:.2f}%".format)
top_school_group['Per Student Budget'] = top_school_group['Per Student Budget'].map("${:.2f}".format)

top_school_group.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,School Type,,,,,,,,
Cabrera High School,Charter,1858,$1081356.00,$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,$1043130.00,$638.00,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,$585858.00,$609.00,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,1468,$917500.00,$625.00,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,2283,$1319574.00,$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
bottom_school_group['Total Students'] = bottom_school_group['Total Students'].map("{:.0f}".format)
bottom_school_group['Per Student Budget'] = bottom_school_group['Per Student Budget'].map("${:.2f}".format)
bottom_school_group['Total School Budget'] = bottom_school_group['Total School Budget'].map("${:.2f}".format)
bottom_school_group['Average Math Score'] = bottom_school_group['Average Math Score'].map("{:.2f}".format)
bottom_school_group['Average Reading Score'] = bottom_school_group['Average Reading Score'].map("{:.2f}".format)    
bottom_school_group['% Passing Math'] = bottom_school_group['% Passing Math'].map("{:.2f}%".format)
bottom_school_group['% Passing Reading'] = bottom_school_group['% Passing Reading'].map("{:.2f}%".format)
bottom_school_group['% Overall Passing Rate'] = bottom_school_group['% Overall Passing Rate'].map("{:.2f}%".format)
bottom_school_group.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,School Type,,,,,,,,
Rodriguez High School,District,3999,$2547363.00,$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,2949,$1884411.00,$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,2917,$1910635.00,$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,4761,$3094650.00,$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,2739,$1763916.00,$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
# creates dataframe and groups by School Name and grade and finds the average score for each grade in each school
math = school_data_complete.loc[:, ['school_name', 'grade', 'math_score']]
math_group = math.groupby(['school_name', 'grade'])
math_group = math_group.mean()
# loops through grouped dataframe and creates lists of each grades averages
nine = []
ten = []
eleven = []
twelve = []
i = 3
while i < 60:
    nine.append(math_group.iat[i,0])
    i = i + 4
i = 0
while i < 60:
    ten.append(math_group.iat[i,0])
    i = i + 4
i = 1
while i < 60:
    eleven.append(math_group.iat[i,0])
    i = i + 4
i = 2
while i < 60:
    twelve.append(math_group.iat[i,0])
    i = i + 4
# creates a final dataframe and adds our lists as columns    
math_scores = math.loc[:, ['school_name', 'math_score']]
math_scores = math_scores.rename(columns={'school_name': 'School Name'})
math_scores = math_scores.groupby('School Name')
math_scores = math_scores.mean()
math_scores['9th'] = nine
math_scores['10th'] = ten
math_scores['11th'] = eleven
math_scores['12th'] = twelve
del math_scores['math_score']
math_scores['9th'] = math_scores['9th'].map("{:.2f}".format)
math_scores['10th'] = math_scores['10th'].map("{:.2f}".format)
math_scores['11th'] = math_scores['11th'].map("{:.2f}".format)
math_scores['12th'] = math_scores['12th'].map("{:.2f}".format)

math_scores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
# creates dataframe and groups by School Name and grade and finds the average score for each grade in each school
reading = school_data_complete.loc[:, ['school_name', 'grade', 'reading_score']]
reading_group = reading.groupby(['school_name', 'grade'])
reading_group = reading_group.mean()
# loops through grouped dataframe and creates lists of each grades averages
nine = []
ten = []
eleven = []
twelve = []
i = 3
while i < 60:
    nine.append(reading_group.iat[i,0])
    i = i + 4
i = 0
while i < 60:
    ten.append(reading_group.iat[i,0])
    i = i + 4
i = 1
while i < 60:
    eleven.append(reading_group.iat[i,0])
    i = i + 4
i = 2
while i < 60:
    twelve.append(reading_group.iat[i,0])
    i = i + 4
# creates a final dataframe and adds our lists as columns    
reading_scores = reading.loc[:, ['school_name', 'reading_score']]
reading_scores = reading_scores.rename(columns={'school_name': 'School Name'})
reading_scores = reading_scores.groupby('School Name')
reading_scores = reading_scores.mean()
reading_scores['9th'] = nine
reading_scores['10th'] = ten
reading_scores['11th'] = eleven
reading_scores['12th'] = twelve
del reading_scores['reading_score']
reading_scores['9th'] = reading_scores['9th'].map("{:.2f}".format)
reading_scores['10th'] = reading_scores['10th'].map("{:.2f}".format)
reading_scores['11th'] = reading_scores['11th'].map("{:.2f}".format)
reading_scores['12th'] = reading_scores['12th'].map("{:.2f}".format)

reading_scores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

spend_bins = unformatted_school_group.loc[:, ["Per Student Budget", 'Average Math Score', 'Average Reading Score', '% Passing Math', 
                          '% Passing Reading', '% Overall Passing Rate']]
spend_bins["Per Student Budget"] = pd.cut(spend_bins["Per Student Budget"], spending_bins, labels=group_names)
spend_bins = spend_bins.groupby("Per Student Budget")
spend_bins = spend_bins.mean()
spend_bins['Average Math Score'] = spend_bins['Average Math Score'].map("{:.2f}".format)
spend_bins['Average Reading Score'] = spend_bins['Average Reading Score'].map("{:.2f}".format)
spend_bins['% Passing Math'] = spend_bins['% Passing Math'].map("{:.2f}%".format)
spend_bins['% Passing Reading'] = spend_bins['% Passing Reading'].map("{:.2f}%".format)
spend_bins['% Overall Passing Rate'] = spend_bins['% Overall Passing Rate'].map("{:.2f}%".format)
spend_bins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
<$585,83.46,83.93,93.46%,96.61%,95.04%
$585-615,83.60,83.89,94.23%,95.90%,95.07%
$615-645,79.08,81.89,75.67%,86.11%,80.89%
$645-675,77.00,81.03,66.16%,81.13%,73.65%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
# Sample bins. Feel free to create your own bins.
school_size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

size_bins = unformatted_school_group.loc[:, ['Total Students', 'Average Math Score', 'Average Reading Score', '% Passing Math', 
                          '% Passing Reading', '% Overall Passing Rate']]
size_bins['Total Students'] = pd.cut(size_bins['Total Students'], school_size_bins, labels=group_names)
size_bins = size_bins.groupby('Total Students')
size_bins = size_bins.mean()
size_bins['Average Math Score'] = size_bins['Average Math Score'].map("{:.2f}".format)
size_bins['Average Reading Score'] = size_bins['Average Reading Score'].map("{:.2f}".format)
size_bins['% Passing Math'] = size_bins['% Passing Math'].map("{:.2f}%".format)
size_bins['% Passing Reading'] = size_bins['% Passing Reading'].map("{:.2f}%".format)
size_bins['% Overall Passing Rate'] = size_bins['% Overall Passing Rate'].map("{:.2f}%".format)
size_bins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Total Students,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,95.20%
Large (2000-5000),77.75,81.34,69.96%,82.77%,76.36%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [9]:

# creates a dataframe grouped by school type with the avg scores and passing rates
type_bins = unformatted_school_group.loc[:, ['School Type', 'Average Math Score', 'Average Reading Score', '% Passing Math', 
                          '% Passing Reading', '% Overall Passing Rate']]
type_bins = type_bins.groupby('School Type')
type_bins = type_bins.mean()
type_bins['Average Math Score'] = type_bins['Average Math Score'].map("{:.2f}".format)
type_bins['Average Reading Score'] = type_bins['Average Reading Score'].map("{:.2f}".format)
type_bins['% Passing Math'] = type_bins['% Passing Math'].map("{:.2f}%".format)
type_bins['% Passing Reading'] = type_bins['% Passing Reading'].map("{:.2f}%".format)
type_bins['% Overall Passing Rate'] = type_bins['% Overall Passing Rate'].map("{:.2f}%".format)
type_bins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%
